<a href="https://colab.research.google.com/github/EvgeniyKuropatkin/-a/blob/main/lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Лабораторная работа 1. Базовое использование LLM

### Инициализируем модель

Qwen2 — новая серия больших языковых моделей Qwen. Для Qwen2 существует ряд базовых языковых моделей и языковых моделей с настройкой инструкций с диапазоном от 0,5 до 72 миллиардов параметров, включая модель «Смесь экспертов». Мы будем использовать настроенную по инструкциям модель 1.5B Qwen2.

По сравнению с современными языковыми моделями с открытым исходным кодом, включая ранее выпущенный Qwen1.5, Qwen2 в целом превзошел большинство моделей с открытым исходным кодом и продемонстрировал конкурентоспособность по сравнению с проприетарными моделями в ряде тестов, нацеленных на понимание языка, создание языков, многоязычные возможности. , кодирование, математика, рассуждения и т. д.

Подробнее с моделью Qwen можно ознакомиться по ссылке https://huggingface.co/Qwen/Qwen2-1.5B-Instruct

In [ ]:
%pip install torch>=2.2.2

In [ ]:
%pip install transformers

In [ ]:
%pip install accelerate

In [ ]:
%pip install nltk

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda"

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-1.5B-Instruct",
    device_map=device,
    torch_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-1.5B-Instruct")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Проверяем работу модели

In [ ]:
prompt = "Коротко расскажи мне о Больших языковых моделях (LLM)."
messages = [
    {"role": "system", "content": "Ты AI помощник Qwen."},
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512,
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Большие языковые модели (или Large Language Models, LLM) - это искусственные интеллекты, которые способны понимать и интерпретировать человеческое язычество на высоком уровне. Они могут выполнять различные задачи, включая перевод текста, анализ текста, генерацию контента, предсказание будущего, анализ данных и многое другое.

Существуют различные типы больших языковых моделей, включая:

1. **Text Generation Models**: Эти модели используются для генерации новых текстов на основе уже существующих.

2. **Question Answering Models**: Они могут ответить на вопросы по заданным вопросам, используя знания из своего обширного объема информации.

3. **Summarization Models**: Их функцией является сжатие большого объема текста до более краткого.

4. **Translation Models**: Они позволяют автоматически перевести текст из одного языка в другой.

5. **Language Modeling Models**: Они используются для прогнозирования следующего слова или предложения в тексте.

6. **Dialog Systems**: Их основная функцион

### 1. Базовый инференс

Напишите функцию, которая будет принимать модель, токенизатор, а также список сообщений пользователя и модели, и возвращать ответ модели в виде текста.

Сообщения должны подаваться в следующем виде
```
chat = [
   {"role": "user", "content": ...},
   {"role": "assistant", "content": ...},
   {"role": "user", "content": ...},
]
```

Полезные функции
https://huggingface.co/docs/transformers/main/en/chat_templating
https://huggingface.co/docs/transformers/en/main_classes/tokenizer

Добавьте `max_new_tokens`, `temperature`, `repetition_penalty`, `no_repeat_ngram_size` в качестве параметров этой функции. Протестируйте работу модели на различных значениях этих параметров.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

def generate_response(model, tokenizer, messages,
                     max_new_tokens=512,
                     temperature=1.0,
                     repetition_penalty=1.0,
                     no_repeat_ngram_size=0,
                     device="cuda" if torch.cuda.is_available() else "cpu"):
    """
    Генерирует ответ модели на основе истории чата.

    Args:
        model: Загруженная языковая модель.
        tokenizer: Токенизатор, соответствующий модели.
        messages (list): Список словарей с ключами 'role' и 'content'.
        max_new_tokens (int): Максимальное количество новых токенов для генерации.
        temperature (float): Параметр температуры для управления случайностью.
        repetition_penalty (float): Штраф за повторение токенов.
        no_repeat_ngram_size (int): Размер n-граммы, которая не должна повторяться.
        device (str): Устройство для вычислений ('cuda' или 'cpu').

    Returns:
        str: Сгенерированный текстовый ответ модели.
    """

    # 1. Применяем шаблон чата для форматирования сообщений
    # tokenize=False возвращает строку, которую мы затем токенизируем вручную
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True  # Добавляем промпт для генерации ответа ассистента
    )

    # 2. Токенизируем отформатированную строку
    # Важно: add_special_tokens=False, так как шаблон уже включает все необходимые специальные токены
    model_inputs = tokenizer([text], return_tensors="pt", add_special_tokens=False).to(device)

    # 3. Генерируем ответ
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        repetition_penalty=repetition_penalty,
        no_repeat_ngram_size=no_repeat_ngram_size,
        do_sample=True if temperature > 0 else False,  # Включаем сэмплирование, если температура > 0
        pad_token_id=tokenizer.eos_token_id  # Устанавливаем pad_token_id для избежания предупреждений
    )

    # 4. Обрезаем входные токены, оставляя только сгенерированный ответ
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    # 5. Декодируем токены обратно в текст
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

# Пример использования и тестирования с различными параметрами
if __name__ == "__main__":
    # Загрузка модели и токенизатора (замените на вашу модель)
    model_name = "Qwen/Qwen2-0.5B-Instruct"  # Пример модели, замените на нужную
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)

    # Определяем историю чата
    chat_history = [
        {"role": "system", "content": "Ты AI помощник Qwen."},
        {"role": "user", "content": "Коротко расскажи мне о Больших языковых моделях (LLM)."}
    ]

    print("=== Тестирование с различными параметрами ===\n")

    # Тест 1: Базовые параметры
    print("Тест 1: Базовые параметры (temperature=1.0)")
    response1 = generate_response(
        model, tokenizer, chat_history,
        max_new_tokens=256,
        temperature=1.0,
        repetition_penalty=1.0,
        no_repeat_ngram_size=0
    )
    print(response1)
    print("\n" + "-"*80 + "\n")

    # Тест 2: Низкая температура (более детерминированный, консервативный ответ)
    print("Тест 2: Низкая температура (temperature=0.3)")
    response2 = generate_response(
        model, tokenizer, chat_history,
        max_new_tokens=256,
        temperature=0.3,
        repetition_penalty=1.0,
        no_repeat_ngram_size=0
    )
    print(response2)
    print("\n" + "-"*80 + "\n")

    # Тест 3: Высокая температура (более креативный, случайный ответ)
    print("Тест 3: Высокая температура (temperature=1.5)")
    response3 = generate_response(
        model, tokenizer, chat_history,
        max_new_tokens=256,
        temperature=1.5,
        repetition_penalty=1.0,
        no_repeat_ngram_size=0
    )
    print(response3)
    print("\n" + "-"*80 + "\n")

    # Тест 4: С штрафом за повторения
    print("Тест 4: Штраф за повторения (repetition_penalty=1.5)")
    response4 = generate_response(
        model, tokenizer, chat_history,
        max_new_tokens=256,
        temperature=1.0,
        repetition_penalty=1.5,
        no_repeat_ngram_size=2
    )
    print(response4)
    print("\n" + "-"*80 + "\n")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

=== Тестирование с различными параметрами ===

Тест 1: Базовые параметры (temperature=1.0)
Большие искусственные лингвистические моделли (LLM) - это специализированные алгоритмы и методы машинного обучения, которые используются для создания искусственных языков. Это могут быть системы, такие как DeepSpeech или Google's Wave Speech Recognition API, или алгоритмы, такие как Natural Language Processing (NLP) или Speech-to-Text.

LLM могут использовать различные методы для создания языков, таких как: 

1. Управление семантическими характеристиками: Они анализируют структуру и контекст языка, чтобы увидеть, какие аспекты важно для создания модели и какие нет.

2. Форматирование и синтаксис: Они могут анализировать языковые правила, чтобы улучшить грамматические и стилистические характеристики.

3. Построение нейронных сетей: Они используются для создания нейронных сетей, которые могут анализировать языковые сущности и создавать модели на основе этих сущностей.

4. Введение в аудио: Они могу

### 2. Суммаризация небольших текстов

До появления LLM задача суммаризации текстов, то есть их краткого пересказа с сохранением смысла считалась сложной задачей. Вот пример из одного курса 2020 года
```
... модели Transformer могут быть использованы для сжатия длинных документов в краткое изложение - задача, известная как суммаризация текста (text summarization). Это одна из самых сложных задач NLP, поскольку она требует целого ряда способностей, таких как понимание длинных отрывков и генерация связного текста, отражающего основные темы документа...
```

Теперь же, с появлением больших языковых моделей суммаризация текста является одной из естественных задач, для которой их применяют. Давайте попробуем суммаризовать отрывок из книги с помощью Qwen2.

In [ ]:
with open('Толстый и тонкий.txt') as f:
    novel = f.read()

print(novel)

Антон Чехов
ТОЛСТЫЙ И ТОНКИЙ
На вокзале Николаевской железной дороги встретились два приятеля: один толстый, другой тонкий. Толстый только что пообедал на вокзале, и губы его, подернутые маслом, лоснились, как спелые вишни. Пахло от него хересом и флер-д'оранжем. Тонкий же только что вышел из вагона и был навьючен чемоданами, узлами и картонками. Пахло от него ветчиной и кофейной гущей. Из-за его спины выглядывала худенькая женщина с длинным подбородком — его жена, и высокий гимназист с прищуренным глазом — его сын.
— Порфирий! — воскликнул толстый, увидев тонкого.— Ты ли это? Голубчик мой! Сколько зим, сколько лет!
— Батюшки! — изумился тонкий.— Миша! Друг детства! Откуда ты взялся?
Приятели троекратно облобызались и устремили друг на друга глаза, полные слез. Оба были приятно ошеломлены.
— Милый мой! — начал тонкий после лобызания.— Вот не ожидал! Вот сюрприз! Ну, да погляди же на меня хорошенько! Такой же красавец, как и был! Такой же душонок и щеголь! Ах ты, господи! Ну, что же ты?

Используйте написанную выше функцию, чтобы получить пересказ рассказа "Толстый и тонкий" в 100 словах.

In [ ]:
prompt = f"""Произведение: «Толстый и тонкий» А. П. Чехова

Кратко перескажи сюжет этого рассказа, сохранив основную идею и ключевые события."""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

summary_ids = model.generate(
    **inputs,
    max_new_tokens=300,
    do_sample=False,
    temperature=0.7,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id
)

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

if summary.startswith(prompt):
    summary = summary[len(prompt):].strip()

print(summary)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


"Толстый и тонкий" - это рассказ о двух людях, которые вместе работают над проектом по созданию телевизионного фильма. Толстый, который является известным актером, занимается режиссурой, а тонкий, который работает в качестве продюсера, занимается сценарием. Они оба стараются сделать фильм, но их подходы и способности различны. 

В конечном итоге, они решают совместно создать новый план, который будет более эффективным и привлекательным для зрителя. Однако, когда они начинают работать над проектом, они столкнулись с множеством препятствий и трудностей. Это включает в себя неудачные моменты, недоразумения со стороны других участников, и даже неприятные ситуации с бюджетом.

После долгого времени работы, они достигли успеха и стали известными благодаря своей работе над фильмом. Но они также понимали, что их работа была не единственной, и они должны были продолжить развивать свои навыки и улучшить качество своих работ. 

Эти два человека, Толстый и тонкий, показывают нам, как важно быть го

### 3. Суммаризация больших текстов

Загрузите текст `Капитанской дочки`.

In [ ]:
with open('Капитанская дочка.txt') as f:
    big_novel = f.read()

print(big_novel[:200], "\n\n", "*"*50, "\n\n", big_novel[-200:])

Александр Пушкин
КАПИТАНСКАЯ ДОЧКА
Береги честь смолоду.
Пословица.

ГЛАВА I
СЕРЖАНТ ГВАРДИИ
— Был бы гвардии он завтра ж капитан.
— Того не надобно; пусть в армии послужит.
— Изрядно сказано! пускай  

 ************************************************** 

 ва эта не включена в окончательную редакцию «Капитанской дочки» и сохранилась в черновой рукописи, где названа «Пропущенная глава». В тексте этой главы Гринев именуется Буланиным, а Зурин — Гриневым.



Узнайте, сколько токенов в указанном произведении. Сравните это число с длиной контекста модели `Qwen2`. Может ли `Qwen2` обработать это произведение за один запуск? Ответ дайте в комментариях

In [ ]:
inputs = tokenizer(big_novel)
print("Токенов в Капитанской дочке:", len(inputs['input_ids']))
print("Длина контекста: ", model.config.max_position_embeddings)

Token indices sequence length is longer than the specified maximum sequence length for this model (81154 > 32768). Running this sequence through the model will result in indexing errors


Токенов в Капитанской дочке: 81154
Длина контекста:  32768


Напишите функцию, которая будет суммаризировать длинный текст, не помещающийся в контекст модели. Для этого длинный текст нужно разбить на короткие кусочки (`chunks`), каждый из которых можно уместить в контексте модели. Чтобы модели было проще уловить контекст, лучше не разбивать предложения. Для этого полезной может быть функция `nltk.sent_tokenize(text)`, которая разбивает текст на предложения.

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:

from nltk.tokenize import sent_tokenize
import torch

# Скачиваем punkt, если ещё не загружен
nltk.download('punkt_tab')  # для новых версий nltk
# или nltk.download('punkt') — в зависимости от окружения

def summarize_long_text(
    text,
    tokenizer,
    model,
    max_tokens_per_chunk=6000,   # оставляем запас под генерацию и служебные токены
    overlap_sentences=2
):
    """
    Суммаризирует длинный текст, разбивая его на чанки по предложениям.

    Args:
        text (str): Исходный текст.
        tokenizer: Токенизатор модели.
        model: Модель для генерации.
        max_tokens_per_chunk (int): Макс. число токенов в чанке.
        overlap_sentences (int): Кол-во перекрывающихся предложений между чанками.

    Returns:
        str: Итоговая суммаризация.
    """
    # Разбиваем на предложения
    sentences = sent_tokenize(text)

    # Формируем чанки
    chunks = []
    current_chunk = []
    current_token_count = 0

    for i, sent in enumerate(sentences):
        # Подсчитываем токены в предложении
        tokens_in_sent = len(tokenizer.encode(sent, add_special_tokens=False))

        # Если добавление предложения превысит лимит — завершаем чанк
        if current_token_count + tokens_in_sent > max_tokens_per_chunk:
            if current_chunk:
                chunks.append(" ".join(current_chunk))
                # Начинаем новый чанк с перекрытия
                current_chunk = current_chunk[-overlap_sentences:] if overlap_sentences > 0 else []
                # Пересчитываем токены для нового чанка
                current_token_count = len(tokenizer.encode(" ".join(current_chunk), add_special_tokens=True))
            else:
                # Очень длинное предложение — добавляем его целиком (редко)
                current_chunk = [sent]
                current_token_count = tokens_in_sent
        else:
            current_chunk.append(sent)
            current_token_count += tokens_in_sent

    # Добавляем последний чанк
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    print(f"Текст разбит на {len(chunks)} чанков.")

    # Суммаризуем каждый чанк
    chunk_summaries = []
    for idx, chunk in enumerate(chunks):
        prompt = f"""Кратко перескажи следующий фрагмент художественного произведения, сохранив ключевые события и персонажей:

{chunk}"""

        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=6100).to(model.device)

        with torch.no_grad():
            summary_ids = model.generate(
                **inputs,
                max_new_tokens=512,
                do_sample=False,
                temperature=0.7,
                top_p=0.9,
                pad_token_id=tokenizer.eos_token_id
            )

        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        # Убираем промпт из вывода
        if summary.startswith(prompt):
            summary = summary[len(prompt):].strip()
        chunk_summaries.append(summary)
        print(f"Суммаризация чанка {idx+1}/{len(chunks)} завершена.")

    # Объединяем и суммаризуем итог
    combined = " ".join(chunk_summaries)

    # Если итог короткий — суммируем напрямую
    if len(tokenizer.encode(combined)) <= 30000:
        final_prompt = f"""На основе следующих фрагментов краткого пересказа составь единый связный и лаконичный пересказ всего произведения:

{combined}"""
    else:
        # Редкий случай — если и суммаризации слишком длинные, можно рекурсивно вызвать функцию
        final_prompt = f"""Суммируй следующий текст, содержащий краткие пересказы частей одного произведения:

{combined}"""

    inputs = tokenizer(final_prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        final_ids = model.generate(
            **inputs,
            max_new_tokens=1024,
            do_sample=False,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    final_summary = tokenizer.decode(final_ids[0], skip_special_tokens=True)
    if final_summary.startswith(final_prompt):
        final_summary = final_summary[len(final_prompt):].strip()

    return final_summary


summary = summarize_long_text(big_novel, tokenizer, model)
print(summary)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Текст разбит на 14 чанков.
Суммаризация чанка 1/14 завершена.
Суммаризация чанка 2/14 завершена.
Суммаризация чанка 3/14 завершена.
Суммаризация чанка 4/14 завершена.
Суммаризация чанка 5/14 завершена.
Суммаризация чанка 6/14 завершена.
Суммаризация чанка 7/14 завершена.
Суммаризация чанка 8/14 завершена.
Суммаризация чанка 9/14 завершена.
Суммаризация чанка 10/14 завершена.
Суммаризация чанка 11/14 завершена.
Суммаризация чанка 12/14 завершена.
Суммаризация чанка 13/14 завершена.
Суммаризация чанка 14/14 завершена.
Эти события происходят в контексте истории, которая основана на событиях, которые происходили в Казани в 1862 году. В этом месте, где Гринев живет, в 1862 году, произошли несколько важных событий, такие как война между Россией и Австро-Венгерской империей, а также конфликты между Россией и Австро-Венгерской империей. Эти события приводят к тому, что Гринев и его семья находятся в опасности, и он должен найти безопасное место для себя.
